Este cuaderno presenta DSPy a través de un sencillo ejemplo de ingeniería de prompts con pocos ejemplos.

### Setting Up

In [1]:
!pip install -U pip
!pip install dspy-ai
!pip install openai~=0.28.1
# !pip install -e $repo_path

import dspy
import sys
import os
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.34.0
    Uninstalling openai-1.34.0:
      Successfully uninstalled openai-1.34.0


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Primeros Pasos
Comenzaremos configurando el modelo de lenguaje (LM). DSPy es compatible con múltiples API y modelos locales. En este cuaderno, trabajaremos con GPT-3.5 (gpt-3.5-turbo).


In [2]:
turbo = dspy.OpenAI(model='gpt-3.5-turbo',api_key ="")

dspy.settings.configure(lm=turbo)


In [3]:
#loading in the data
!git clone https://github.com/skandavivek/transformerQA-finetuning.git

Cloning into 'transformerQA-finetuning'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (13/13), 2.41 MiB | 4.79 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [5]:
import pandas as pd
df = pd.read_csv('./train.csv')


In [6]:
#as you can see, there are 2 types of answers - one is ANSWERNOTFOUND, the other is when the extracted answer is present
df['human_ans_spans'].astype(str)

0                                      ANSWERNOTFOUND
1                                      ANSWERNOTFOUND
2                                      ANSWERNOTFOUND
3                            this show is OUTSTANDING
4       The costume design by Susan Matheson is great
                            ...                      
2496                                   ANSWERNOTFOUND
2497                                   ANSWERNOTFOUND
2498                                   ANSWERNOTFOUND
2499                                   ANSWERNOTFOUND
2500                                   ANSWERNOTFOUND
Name: human_ans_spans, Length: 2501, dtype: object

In [7]:
df1=df.loc[df['human_ans_spans']=='ANSWERNOTFOUND'].reset_index(drop=True)
df2=df.loc[df['human_ans_spans']!='ANSWERNOTFOUND'].reset_index(drop=True)


In [8]:
df1=df1[:4][['question','review','human_ans_spans']]
df2=df2[:4][['question','review','human_ans_spans']]

df1.columns=['question','context','answer']

df2.columns=['question','context','answer']
df_c=pd.concat([df1,df2]).sort_index().reset_index(drop=True)
df_c


,question,context,answer
0,Who is the author of this series?,Whether it be in her portrayal of a nerdy lesb...,ANSWERNOTFOUND
1,Is this series good and excelent?,"At the time of my review, there had been 910 c...",this show is OUTSTANDING
2,Can we enjoy the movie along with our family ?,"An outstanding romantic comedy, 13 Going on 30...",ANSWERNOTFOUND
3,How is the costume design?,"""Fright Night"" is great! This is how the story...",The costume design by Susan Matheson is great
4,Does this one good?,"To let the truth be known, I watched this movi...",ANSWERNOTFOUND
5,What criticism deserves the movie Passion of C...,Revenge of the Sith is for children and for ad...,Oh my god is this a STUPID film
6,How are the special effects?,As with all Star Wars (or science fiction) mov...,ANSWERNOTFOUND
7,How is the scene?,The best KingKong movie ever made in my opinio...,KingKong


In [9]:
qa_pairs = json.loads(df_c.to_json(orient="records"))

In [10]:
qa_pairs[0]

{'question': 'Who is the author of this series?',
 'context': "Whether it be in her portrayal of a nerdy lesbian or a punk rock rebel, Maslany's plural personalities, (though very stereotypical), are entertaining eye-candy. Combined with a complex and unpredictable plot line, this show is surprisingly addictive. ANSWERNOTFOUND",
 'answer': 'ANSWERNOTFOUND'}

In [11]:
# Create dataset
dataset = [dspy.Example(x).with_inputs('question','context') for x in qa_pairs]

trainset = [x.with_inputs('question','context') for x in dataset[:4]]
devset = [x.with_inputs('question','context') for x in dataset[4:]]

In [12]:
train_example = trainset[0]
print(f"Question: {train_example.question}")
print(f"Context: {train_example.context}")
print(f"Answer: {train_example.answer}")

Question: Who is the author of this series?
Context: Whether it be in her portrayal of a nerdy lesbian or a punk rock rebel, Maslany's plural personalities, (though very stereotypical), are entertaining eye-candy. Combined with a complex and unpredictable plot line, this show is surprisingly addictive. ANSWERNOTFOUND
Answer: ANSWERNOTFOUND


In [13]:
dev_example = devset[0]


Después de cargar los datos en bruto, aplicamos x.with_inputs('question') a cada ejemplo para indicarle a DSPy que nuestro campo de entrada en cada ejemplo será únicamente question. Cualquier otro campo son etiquetas o metadatos que no se proporcionan al sistema.

In [14]:
print(f"For this dataset, training examples have input keys {train_example.inputs().keys()} and label keys {train_example.labels().keys()}")
print(f"For this dataset, dev examples have input keys {dev_example.inputs().keys()} and label keys {dev_example.labels().keys()}")

For this dataset, training examples have input keys ['question', 'context'] and label keys ['answer']
For this dataset, dev examples have input keys ['question', 'context'] and label keys ['answer']


### Bloques de Construcción
Después de cargar los datos en bruto, aplicamos x.with_inputs('question') a cada ejemplo para indicarle a DSPy que nuestro campo de entrada en cada ejemplo será únicamente question. Cualquier otro campo son etiquetas o metadatos que no se proporcionan al sistema.

##### Uso del Modelo de Lenguaje: Firmas y Predictores
Cada llamada al modelo de lenguaje (LM) en un programa de DSPy necesita tener una Firma.

Una firma consta de tres elementos simples:

Una descripción mínima de la sub-tarea que el LM debe resolver.
Una descripción de uno o más campos de entrada (por ejemplo, la pregunta de entrada) que proporcionaremos al LM.
Una descripción de uno o más campos de salida (por ejemplo, la respuesta a la pregunta) que esperaremos del LM.
Definamos una firma simple para la respuesta básica a preguntas.

In [18]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

En BasicQA, el docstring describe la sub-tarea aquí (es decir, responder preguntas). Cada InputField o OutputField puede contener opcionalmente una descripción desc también. Cuando no se proporciona, se infiere del nombre del campo (por ejemplo, question).

Observa que no hay nada especial sobre esta firma en DSPy. Podemos definir fácilmente una firma que tome un fragmento largo de un PDF y devuelva información estructurada, por ejemplo.

De todos modos, ahora que tenemos una firma, definamos y usemos un Predictor. Un predictor es un módulo que sabe cómo usar el LM para implementar una firma. Lo más importante, los predictores pueden aprender a ajustar su comportamiento a la tarea.

In [17]:
# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

# Call the predictor on a particular input.
pred = generate_answer(question=dev_example.question,context = dev_example.context)

# Print the input and the prediction.
print(f"Question: {dev_example.question}")
print(f"Predicted Answer: {pred.answer}")

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

In [19]:
turbo.inspect_history(n=1)

''

In [20]:
# Define the predictor. Notice we're just changing the class. The signature BasicQA is unchanged.
generate_answer_with_chain_of_thought = dspy.ChainOfThought(BasicQA)

# Call the predictor on the same input.
pred = generate_answer_with_chain_of_thought(question=dev_example.question)

# Print the input, the chain of thought, and the prediction.
print(f"Question: {dev_example.question}")
print(f"Thought: {pred.rationale.split(':', 1)[1].strip()}")
print(f"Predicted Answer: {pred.answer}")

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

In [21]:
pred.rationale

NameError: name 'pred' is not defined

### Programa 1: Ejemplo Básico con Pocos Ejemplos
Definamos nuestro primer programa completo para esta tarea. Construiremos una canalización de pocos ejemplos para la generación de respuestas.

Comencemos definiendo esta firma: context, question --> answer.

In [22]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

### Definiendo el Programa
Vamos a definir el programa actual. Esto es una clase que hereda de dspy.Module.

Necesita dos métodos:

El método __init__ declarará los sub-módulos que necesita: dspy.Retrieve y dspy.ChainOfThought. Este último está definido para implementar nuestra firma GenerateAnswer.
El método forward describirá el flujo de control para responder la pregunta utilizando los módulos que tenemos.

In [23]:
class fs(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        #self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question,context):
        #context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

### Compilando el programa de pocos ejemplos
Habiendo definido este programa, ahora vamos a compilarlo. Compilar un programa actualizará los parámetros almacenados en cada módulo. En nuestro caso, esto consiste principalmente en recolectar y seleccionar buenas demostraciones para incluir en tus prompts.

Compilar depende de tres cosas:

Un conjunto de entrenamiento. Usaremos nuestros 20 ejemplos de preguntas y respuestas del trainset mencionado anteriormente.
Una métrica para la validación. Definiremos rápidamente validate_answer que verifica si la respuesta predicha es correcta. También verificará que el contexto recuperado realmente contenga esa respuesta.
Un teleprompter específico. El compilador de DSPy incluye varios teleprompters que pueden optimizar tus programas.

Teleprompters: Los teleprompters son poderosos optimizadores que pueden tomar cualquier programa y aprender a seleccionar y mejorar de manera efectiva los prompts para sus módulos. De ahí el nombre, que significa "prompting a distancia".

Diferentes teleprompters ofrecen varios equilibrios en términos de cuánto optimizan el costo versus la calidad, etc. Usaremos un sencillo teleprompter por defecto BootstrapFewShot en este cuaderno.

Si te gustan las analogías, podrías pensar en esto como tus datos de entrenamiento, tu función de pérdida y tu optimizador en una configuración estándar de aprendizaje supervisado con redes neuronales profundas (DNN). Mientras que SGD es un optimizador básico, existen otros más sofisticados (¡y más costosos!) como Adam o RMSProp.

In [24]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    #answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM

# Set up a basic teleprompter, which will compile our program.
teleprompter = BootstrapFewShot(metric=validate_answer)

# Compile!
compiled_fs = teleprompter.compile(fs(), trainset=trainset)

  0%|          | 0/4 [00:00<?, ?it/s]2024-06-18T12:01:17.218143Z [error    ] Failed to run or to evaluate example Example({'question': 'Who is the author of this series?', 'context': "Whether it be in her portrayal of a nerdy lesbian or a punk rock rebel, Maslany's plural personalities, (though very stereotypical), are entertaining eye-candy. Combined with a complex and unpredictable plot line, this show is surprisingly addictive. ANSWERNOTFOUND", 'answer': 'ANSWERNOTFOUND'}) (input_keys={'question', 'context'}) with <function validate_answer at 0x7110b3b5d870> due to No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.. [dspy.teleprompt.bootstrap] filename=b

# Probando el Programa Compilado
Ahora que hemos compilado nuestro programa, es momento de probarlo. Para ello, utilizaremos algunos ejemplos de entrada y verificaremos si el modelo proporciona respuestas correctas.

In [25]:
# Ask any question you like to this simple few-shot program.


my_question = "Do you like Avocados?"
context = dev_example.context

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_fs(my_question,context)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts: {pred.context}")

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

Para inspeccionar el último prompt utilizado para el modelo de lenguaje (LM), podemos modificar el método forward de la clase BasicFewShotProgram para imprimir el prompt antes de enviarlo al LM. Esto nos permitirá ver exactamente qué prompt se está utilizando para generar la respuesta.

In [26]:
turbo.inspect_history(n=1)

''